# Optimize with Hyperactive
[Home](https://simonblanke.github.io/hyperactive-documentation/)

## Install


In [ ]:
!pip install hyperactive

## Sample strategy

In [1]:
import talib.abstract as ta

from lettrade import DataFeed, Strategy, crossover, crossunder
from lettrade.exchange.backtest import ForexBackTestAccount, let_backtest


class SmaCross(Strategy):
    ema1_period = 9
    ema2_period = 21

    def indicators(self, df: DataFeed):
        df["ema1"] = ta.EMA(df, timeperiod=self.ema1_period)
        df["ema2"] = ta.EMA(df, timeperiod=self.ema2_period)

        df["signal_ema_crossover"] = crossover(df.ema1, df.ema2)
        df["signal_ema_crossunder"] = crossunder(df.ema1, df.ema2)

    def next(self, df: DataFeed):
        if len(self.orders) > 0 or len(self.trades) > 0:
            return

        if df.signal_ema_crossover[-1]:
            price = self.data.close[-1]
            self.buy(size=0.1, sl=price - 0.01, tp=price + 0.01)
        elif df.signal_ema_crossunder[-1]:
            price = self.data.close[-1]
            self.sell(size=0.1, sl=price + 0.01, tp=price - 0.01)

lt = let_backtest(
    strategy=SmaCross,
    datas="example/data/data/EURUSD_5m_0_10000.csv",
    account=ForexBackTestAccount,
)

### Example

In [3]:
from hyperactive import Hyperactive
from hyperactive.optimizers import HillClimbingOptimizer

# define the model in a function
def params_parser(args):
    return [
        ("ema1_period", int(args["ema1_period"])),
        ("ema2_period", int(args["ema2_period"])),
    ]


def result_parser(result):
    return result["Equity [$]"]


model = lt.optimize_instance(
    params_parser=params_parser,
    result_parser=result_parser,
    fork_data=True,
)

# search space determines the ranges of parameters you want the optimizer to search through
search_space = {
    "ema1_period": list(range(5, 25, 1)),
    "ema2_period": list(range(10, 50)),
}

optimizer = HillClimbingOptimizer(epsilon=0.1, distribution="laplace", n_neighbours=4)

# start the optimization run
hyper = Hyperactive()
hyper.add_search(model, search_space, optimizer=optimizer, n_iter=100)
hyper.run()

[0] _optimize_instance (Hill Climbing):   0%|          | 0/100 [00:00<?, ?it/s]

[0] _optimize_instance (Hill Climbing): 100%|──────────| 100/100 [00:01<00:00, 50.75it/s, best_iter=0, best_pos=[10 27], best_score=1063.8] 



Results: '_optimize_instance'  
   Best score: 1063.8  
   Best parameter set:
      'ema1_period' : 15.0  
      'ema2_period' : 37.0  
   Best iteration: 0  
 
   Random seed: 1231760450  
 
   Evaluation time   : 1.8575713634490967 sec    [99.02 %]
   Optimization time : 0.018411874771118164 sec    [0.98 %]
   Iteration time    : 1.8759832382202148 sec    [53.31 iter/sec]
 
